# OR Advent 2024 December 1: meeting rooms (Kotlin)

## Assignment

Hi, I'm ÉléonORe, and I'm struggling to organize a series of events...

Each event requires a dedicated room.

Some events have overlapping participants, so I can't schedule them in the same room at the same time.

How many rooms do I need? How can I assign each event to a room so that no two overlapping events are scheduled in the same one?

Ideally, I want to minimize the total number of rooms used.

Can you help me solve this problem?

## Note

**This implementation presumes each event needs to be scheduled into a room and into a timeslot.**
This differs from the variant with only 1 timeslot for which participants can attend multiple meetings in that one timeslot.

The number of timeslots in limited to a single day of 8 meeting hours with 30 minutes per meeting, so 16 timeslots in total:

In [1]:
val timeslotCount = 16
val maxRoomCount = 20


## Dependencies

This Kotlin Jupyter Notebook solves a school timetabling problem with [Timefold Solver](https://timefold.ai/open-source-solver), the open source planning solver AI. We add it as a dependency:

In [2]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.16.0")


## Domain

We need to assign events to timeslots and rooms:

In [3]:
import java.time.LocalDateTime

data class Timeslot(
    val start: LocalDateTime
)

In [4]:
data class Room(
    val id: String
)

This class changes during planning. Timefold Solver fills in the `@PlanningVariable ` annotated fields for all instances:

In [5]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.lookup.PlanningId
import ai.timefold.solver.core.api.domain.variable.PlanningVariable

@PlanningEntity
data class Event(
    @PlanningId val id: String) {
    
    val conflictingEvents = mutableSetOf<Event>()

    @PlanningVariable
    var timeslot: Timeslot? = null
    @PlanningVariable
    var room: Room? = null

    // No-arg constructor required for Timefold
    constructor() : this("")
}

## Constraints

The solver takes into account hard and soft constraints:

In [6]:
import ai.timefold.solver.core.api.score.buildin.hardmediumsoftlong.HardMediumSoftLongScore
import ai.timefold.solver.core.api.score.stream.*
import java.time.Duration

class OrAdventConstraintProvider : ConstraintProvider {

    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint>? {
        return arrayOf(
            // Hard constraints
            roomConflict(constraintFactory),
            attendeeConflict(constraintFactory),
            // Soft constraints
            minimizeRooms(constraintFactory),
            antiLoadBalanceRooms(constraintFactory)
        )
    }

    fun roomConflict(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEachUniquePair(
                Event::class.java,
                Joiners.equal(Event::timeslot),
                Joiners.equal(Event::room)
            )
            .penalize(HardMediumSoftLongScore.ofHard(1000))
            .asConstraint("Room conflict");
    }

    fun attendeeConflict(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEachUniquePair(
                Event::class.java,
                Joiners.equal(Event::timeslot),
                Joiners.filtering{event1: Event, event2: Event -> event1.conflictingEvents.contains(event2)}
            )
            .penalize(HardMediumSoftLongScore.ONE_HARD)
            .asConstraint("Attendee conflict");
    }

    fun minimizeRooms(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEach(Event::class.java)
            .groupBy(Event::room)
            .penalize(HardMediumSoftLongScore.ONE_MEDIUM)
            .asConstraint("Minmize rooms");
    }

    fun antiLoadBalanceRooms(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEach(Event::class.java)
            .groupBy(Event::room, ConstraintCollectors.count())
            .reward(HardMediumSoftLongScore.ONE_SOFT) {room, count -> count * count}
            .asConstraint("Anti load balance rooms");
    }

}

### The dataset class

This class represents the dataset to solve:

In [7]:
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.score.buildin.hardmediumsoftlong.HardMediumSoftLongScore

@PlanningSolution
data class OrAdventDataset(
    
    @ValueRangeProvider
    val timeslots: List<Timeslot>,
    @ValueRangeProvider
    val rooms: List<Room>,
    @PlanningEntityCollectionProperty
    val events: List<Event>) {

    @PlanningScore
    var score: HardMediumSoftLongScore? = null

    // No-arg constructor required for Timefold
    constructor() : this(emptyList(), emptyList(), emptyList())

}

## Input reader

In [8]:
import java.io.File

fun readDataset(): OrAdventDataset {
    val timeslots = mutableListOf<Timeslot>()
    var nextDateTime = LocalDateTime.of(2024,12,25,9,0)
    for (i in 1..timeslotCount) {
        timeslots.add(Timeslot(nextDateTime))
        nextDateTime = nextDateTime.plusMinutes(30)
    }
    val rooms = mutableListOf<Room>()
    for (i in 1..maxRoomCount) {
        rooms.add(Room("$i"))
    }

    val input: String = File("./instance.txt").readText()
    val lines = input.lines().filter {
        it.isNotBlank() && !it.startsWith("#") // Skip empty or commented lines
    }
    val (numberOfEvents, _) = lines.first().split(" ").map { it.toInt() }
    val events = mutableListOf<Event>()
    for (i in 1..numberOfEvents) {
        events.add(Event("$i"))
    }
    val eventsMap = events.map { it.id to it }.toMap()
    
    lines.drop(1).forEach { line ->
        val (_, eventId1, eventId2) = line.split(" ")
        val event1 = eventsMap[eventId1]!!
        val event2 = eventsMap[eventId2]!!
        
        event1.conflictingEvents.add(event2)
        event2.conflictingEvents.add(event1)
    }
    return OrAdventDataset(timeslots, rooms, events)
}

## Solve it

Configure and run the solver:

In [9]:
import ai.timefold.solver.core.config.solver.SolverConfig
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.Solver

val solverFactory: SolverFactory<OrAdventDataset> = SolverFactory.create(SolverConfig()
        .withSolutionClass(OrAdventDataset::class.java)
        .withEntityClasses(Event::class.java)
        .withConstraintProviderClass(OrAdventConstraintProvider::class.java)
        // The solver runs only for 5 seconds on this small dataset.
        // It's recommended to run for at least 5 minutes ("5m") otherwise.
        .withTerminationSpentLimit(Duration.ofSeconds(10)))

val problem: OrAdventDataset = readDataset()

println("Solving the problem ...")
val solver: Solver<OrAdventDataset> = solverFactory.buildSolver()
val solution: OrAdventDataset = solver.solve(problem)
println("Solving finished with score (${solution.score}).")

Solving the problem ...
Solving finished with score (0hard/-7medium/1510soft).


## Visualize the solution

In [10]:
val eventByRoomMap = solution.events.groupBy { Pair(it.timeslot, it.room) }
HTML(buildString {
    append("<p style='font-size: x-large'>Score: ${solution.score}</p>")
    append("<p><b>By room</b></p>")
    append("<table><tr><th/>")
    for (room in solution.rooms) {
        append("<th>${room.id}</th>")
    }
    append("</tr>")
    for (timeslot in solution.timeslots) {
        append("<tr><th>${timeslot.start}</th>")
        for (room in solution.rooms) {
            val cellEvents = eventByRoomMap.get(Pair(timeslot, room)) ?: emptyList()
            append("<td>")
            append(cellEvents.map { it.id }.joinToString(", "))
            append("</td>")
        }
        append("</tr>")
    }
    append("</table>")
})

Score: 0hard/-7medium/1510soft By room 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 2024-12-25T09:00 47 96 80 87 27 2024-12-25T09:30 97 69 12 32 99 37 65 2024-12-25T10:00 14 5 17 39 61 95 2024-12-25T10:30 100 70 38 66 6 18 2024-12-25T11:00 68 43 48 30 16 21 2024-12-25T11:30 76 91 13 49 82 85 2024-12-25T12:00 73 74 2 1 64 92 77 2024-12-25T12:30 4 15 93 98 81 9 45 2024-12-25T13:00 54 11 88 72 83 19 8 2024-12-25T13:30 67 55 33 22 53 84 2024-12-25T14:00 50 24 28 44 23 62 2024-12-25T14:30 10 42 59 63 56 2024-12-25T15:00 58 71 25 41 7 40 31 2024-12-25T15:30 34 60 35 78 57 46 2024-12-25T16:00 86 90 51 20 29 3 36 2024-12-25T16:30 75 89 26 94 79 52

## Statistics

For a big dataset, a schedule visualization is often too verbose.
Let's visualize the solution through statistics:

In [11]:
%use kandy

In [12]:
val roomIds = solution.rooms.map { it.id }
val eventCounts = solution.rooms.map { room -> solution.events.filter{ it.room == room }.count() }

plot {
    layout.title = "Events per room"
    bars {
        x(roomIds) { axis.name = "Rooms" }
        y(eventCounts) { axis.name = "Events" }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="VvhhDm"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Events per room"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Rooms"
},{
"aesthetic":"y",
"name":"Events",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20"],
"y":[0.0,0.0,15.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,16.0,0.0,6.0,16.0,15.0,0.0]
},
"sampling":"none",
"position":"dodge",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("VvhhDm");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 
 
 9 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 11 
 
 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 
 
 13 
 
 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 16 
 
 
 
 
 
 
 
 
 17 
 
 
 
 
 
 
 
 
 18 
 
 
 
 
 
 
 
 
 19 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 16 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Events per room 
 
 
 
 
 Events 
 
 
 
 
 Rooms

## Analyze the score

Let's break down the score per constraint:

In [13]:
import ai.timefold.solver.core.api.solver.SolutionManager

val solutionManager = SolutionManager.create(solverFactory)
val scoreAnalysis = solutionManager.analyze(solution)

And visualize it:

In [14]:
HTML(buildString {
    append("<p style='font-size: x-large'>Score: ${scoreAnalysis.score}</p>")
    append("<ul>")
    for (constraint in scoreAnalysis.constraintMap().values) {
        append("<li>${constraint.constraintRef().constraintName}: ${constraint.score.toShortString()}</li>")
    }
    append("</ul>")
})

Score: 0hard/-7medium/1510soft Anti load balance rooms: 1510soft Minmize rooms: -7medium Attendee conflict: 0 Room conflict: 0

## Conclusion

To learn more about planning optimization, visit [timefold.ai](https://timefold.ai).